# Week 8: Interactive Visualizations

## a Brief Recap:

* Hello, how are you?
* Today....split in 2!:
    - Interactive Visualizations
    - Python Classes
* Next Week We will get started with Machine Learning basics
* A thing or two about Homework 1
* Any questions so far about Homework 2?

## Interactive Visualizations with Python

Using the Psychophysics DataFrame that results from Homework2:

* Plotly
* Dash Apps
* Streamlit Apps

there are some other libraries that specialize in [interactive plots](https://mode.com/blog/python-interactive-plot-libraries/) if you are curious

## The SCSF Psychophysics Data

Let's do a brief EDA and some visualizations

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
url = 'https://raw.githubusercontent.com/SmilodonCub/DS4VS/master/Homework/SCSF.csv'
SCSF_df = pd.read_csv( url, converters={'scase1_reversals': eval,'scase2_reversals': eval}  )
SCSF_df.shape

In [ ]:
SCSF_df.head()

## Some light Preprocessing

Let's functionalize some of the code we used last week to find a rough threshold for each staircase

In [ ]:
def addRoughThreshold( data_df, col_list ):
    """
    given a dataframe and a list of names for staircase reversal columns,
    return a dataframe with a feature added giving the rough threshold for each record.
    assumes each record hold staircases with identical settings
    """
    df = data_df.copy()
    
    sc1tail = pd.Series( [ x[-4:] for x in df[col_list[0]] ] )
    sc2tail = pd.Series( [ x[-4:] for x in df[col_list[1]] ] )
    df['reversal'] = sc1tail + sc2tail
    return df
    

In [ ]:
col_names = [ 'scase1_reversals', 'scase2_reversals' ]
df = addRoughThreshold( SCSF_df, col_names )
df.head()

## subjects & conditions

In [ ]:
print( df.observer_name.unique() )
print( df.type_grating.unique() )
print( df.spatial_frequency.unique() )
print( df.contrast_level.unique() )
print( df.rgb_min.unique() )
print( df.rgb_max.unique() )

## A basic visualization

In [ ]:
# groupby spatial frequency & subject
df = df.groupby(['observer_name','spatial_frequency', 'type_grating']).agg({'reversal':'sum'})
df['reversal_mean'] = [ np.array( x ).mean() for x in df['reversal'] ]
df['sensitivity'] = 1/df['reversal_mean']
df.head()

In [ ]:
splot = sns.relplot( data = df, x = 'spatial_frequency', y = 'sensitivity', 
            hue = 'type_grating', col = 'observer_name', kind = 'line', col_wrap=2 )
splot.set(xscale="log")
plt.show()

## Plotly

[Plotly](https://plotly.com/graphing-libraries/) is a family of graphing libraries that can be used to build interactive visualizations in Python, R, Julia and Matlab  

[Python Plotly Gallery](https://plotly.com/python/)

<img src="https://upload.wikimedia.org/wikipedia/commons/8/8a/Plotly_logo_for_digital_final_%286%29.png" width="40%" style="margin-left:auto; margin-right:auto">

## Plotly as basic visualization

Plotly can be used for basic visualization.  
Here we recreate the same basic image with the SCSF data:

In [ ]:
df2 = df.reset_index()
df2.head()

In [ ]:
import plotly.express as px
# themes = ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]
fig = px.line( df2, x ='spatial_frequency', y ='sensitivity', color ='type_grating', 
                 facet_col = 'observer_name', facet_col_wrap=2, log_x=True, template= 'plotly_white')
fig.show()

## Plotly + Dash for Interactive mini-Apps

Build complex, interactive web-based dashboards/applications  

Dash is a library 'built on top of' Plotly and [Flask](https://flask.palletsprojects.com/en/2.0.x/).  
Flask is a popular Python web-framework, but setting it up can be tricky.  
Dash makes deploying apps much simpler!  

Let's use the SFSC data to build an example....

## Subject Results Dashboard

We would like to use Dash to build an interactive web-application that will allow us to view the results of each subject

In [ ]:
import dash
from dash import dcc, html 
from dash.dependencies import Input, Output, State

## Dash App Components

1. the data ...and  dream (the information you want to involve | idea you would like to convey)
2. the `app.layout` - describes what the application looks like
3. the `app.callback` - describes the interactivity

We can do simple apps in Jupyter, but ideally the app.layout and app.callback would be seperate files in an app root directory

### data|dream

Exactly what do we want to render?  
Spend some time conceptualizing before you start building an app.  
You will save yourself a lot of time if you organize your thoughts first!

In [ ]:
app = dash.Dash(__name__)

available_subjects = df2.observer_name.unique()
df3 = df2[ df2[ 'observer_name' ] == 'kh' ]
fig = px.line( df3, x ='spatial_frequency', y ='sensitivity', color ='type_grating', 
              log_x=True, template= 'plotly_white' )

Sketch your layout!...how should the data be organized?

<img src="https://raw.githubusercontent.com/SmilodonCub/DS4VS/master/Week8/sketch.png" width="40%" style="margin-left:auto; margin-right:auto">



### app.layout

We've already spent time organizing our data, so let's dive into the layout

In [ ]:
app.layout = html.Div([
    html.H1('SCSF Subject Sensitivity'),
    html.Div([
        html.Div([
            dcc.Dropdown(
                id='Observer',
                options=[{'label': i, 'value': i} for i in available_subjects],
                value='kh'
            )
        ], style={'width': '25%', 'display': 'inline-block'}),
    ]),
    dcc.Graph(
        id='subject_graph',
        figure=fig
    )

])

if __name__ == '__main__':
    app.run_server(debug=False)

### app.callback

We were able to view a subject's results and the graph appeared as expected. However, we were unable to interact with the visualization to select a different subject.  

The interactivity is controlled by the app.callback which we will now implement....

In [ ]:
@app.callback(
    Output('subject_graph' ),
    Input('Observer', 'value'))

def update_graph(value):
    df3 = df2[ df2[ 'observer_name' ] == value ]
    fig = px.line( df3, x ='spatial_frequency', y ='sensitivity', color ='type_grating', 
              log_x=True, template= 'plotly_white' )
    return fig

### Now to sew these pieces together...

In [ ]:
app = dash.Dash(__name__)

available_subjects = df2.observer_name.unique()
df3 = df2[ df2[ 'observer_name' ] == 'kh' ]
fig = px.line( df3, x ='spatial_frequency', y ='sensitivity', color ='type_grating', 
              log_x=True, template= 'plotly_white' )


app.layout = html.Div([
    html.H1('SCSF Subject Sensitivity'),
    html.Div([
        html.Div([
            dcc.Dropdown(
                id='Observer',
                options=[{'label': i, 'value': i} for i in available_subjects],
                value='kh'
            )
        ], style={'width': '25%', 'display': 'inline-block'}),
    ]),
    dcc.Graph(
        id='subject_graph',
        figure=fig )
])

@app.callback(
    Output('subject_graph', 'figure' ),
    Input('Observer', 'value'))

def update_graph(value):
    df3 = df2[ df2[ 'observer_name' ] == value ]
    fig = px.line( df3, x ='spatial_frequency', y ='sensitivity', color ='type_grating', 
              log_x=True, template= 'plotly_white' )
    return fig

if __name__ == '__main__':
    app.run_server(debug=False)

### App Deployment

Well that's great! In relatively few lines of code we pulled together an interactive figure to view some results of a psychophysical experiment.  
**Problem**: right now the visualization only works in the browser on our local machine.  

To create a sharable link, we need to deploy the app

for this we need a production ready HTTP server. ....  
this is out of the scope for this lecture. However, there are some [great resources](https://towardsdatascience.com/deploying-your-dash-app-to-heroku-the-magical-guide-39bd6a0c586c) that can help you if you are interested and have the time.


<img src="https://blog.talenox.com/wp-content/uploads/2015/01/Heroku_logo.svg.png" width="40%" style="margin-left:auto; margin-right:auto">

### Deploying with Heroku

0. Have Github & Heroku accounts (you're halfway there!)
1. Create a new folder for your app
2. Initialize your folder with both `git` and `virtualenv`
3. Install `gunicorn` and all your app's dependancies (`virtualenv` creates a fresh python environment)
4. Initialize a folder with:
    - app.py
    - .gitignore
    - Procfile
    - requirements.txt
5. Initialize Heroku, add the files to git and deploy!

### A Shareable App!

[SCSF Subject Sensitivity](https://glacial-retreat-76947.herokuapp.com/)

Deployment instructions:

* [Dash Documentation](https://dash.plotly.com/deployment)  
* [Heroku Deploying with Git](https://devcenter.heroku.com/articles/git)
* `$ heroku logs --tail` -for error logs
* breath! and keep working. You will figure it out.

## Streamlit

Streamlit is all about speed & interaction  
Streamlit doesn't mess with html, it is Python-centric
No callbacks: 'widget' objects are treated as variables  
Data caching speeds up rendering

In [ ]:
pip install streamlit

### Organization for Streamlit

Best practice:

Have a dedicated folder for steamlit apps. e.g.: `streamlit_apps`  
Within `streamlit-apps` have a different folder for each app you develop  

For this class, let's set up:

    streamlit_apps
        ↳SCSF_app

### Setup via Terminal

1) in terminal, navigate to the directory you'd like to place your `streamlit_apps` folder

    mkdir streamlit_apps
    cd streamlit_apps
    
2) in terminal, create an new folder for our SCSF app, change directories to the new folder and create a new file:

    mkdir SCSF_app
    cd SCSF_app
    touch SCSF_demo.py

3) in a text editor, open `SCSF_demo.py`, paste the following code and save your changes....   

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

st.title( 'SCSF Subject Sensitivity' )
url = 'https://raw.githubusercontent.com/SmilodonCub/DS4VS/master/Week8/SCSF_df'
SCSF_df = pd.read_csv( url )
st.write( SCSF_df.head() )

st.markdown( 'Use this app to select a subject of interest and view their SCSF results' )
available_subjects = SCSF_df.observer_name.unique()
selected_subject = st.selectbox('Which subject?', available_subjects )
sub_df = SCSF_df[ SCSF_df[ 'observer_name' ] == selected_subject ]

fig, ax = plt.subplots()
ax = sns.lineplot( data = sub_df, x = 'spatial_frequency', y = 'sensitivity', 
            hue = 'type_grating' )
ax.set(xscale="log")
st.pyplot( fig )

4) Great!, now in terminal type:

    streamlit run SCSF_demo.py
    
    
You should see an interactive figure in your browser (localhost)
   

That was relatively painless wasn't it!  

Let's see how easy it is to make changes to a local Streamlit app....  

* add x & y labels to the figure
* add a title

In the upper right select 'Rerun'

## Deploying Streamlit App to Heroku

**Question** - Do we want to try to deploy our app? ....we could move on to other things...

In a text editor we need to make 3 new files and save to app folder:  

1. Procfile
2. setup.sh
3. requirements.txt

### Procfile

    web: sh setup.sh && streamlit run SCSF_demo.py
    
note: if you have a different file name instead of `SCSF_demo.py` please replace

### setup.sh

    mkdir -p ~/.streamlit/
    echo "\
    [server]\n\
    headless = true\n\
    port = $PORT\n\
    enableCORS = false\n\
    \n\
    " > ~/.streamlit/config.toml

### requirements.txt

    matplotlib==3.4.3
    numpy==1.19.5
    pandas==1.3.3
    seaborn==0.11.1
    streamlit==0.89.0
    
note: if you make changes and need any other libraries please add

### In Terminal:

Initialize git, add and commit

    git init
    git add .
    git commit -m 'SCSF first commit'
    
Initialize heroku
    
    heroku create

### In your Heroku account:

you should see an update in your account dashboard. (e.x. obscure-dawn-74721)  
copy the name of the new instance in your account

### back in Terminal:

    heroku git:remote -a yourapp
    git push heroku master

[Et Voila!](https://obscure-dawn-74721.herokuapp.com/)

## Discussion

Deep breath! Think of something calm & soothing. That was a lot to take in!  

Let's discuss the process of deploying an interactive app:

* Can you imagine a use for this in your lab? Explain...

<img src="https://ichef.bbci.co.uk/images/ic/640x360/p07j42z8.jpg" width="40%" style="margin-left:auto; margin-right:auto">


## Wow! You made it through this notebook. You deserve a pat on the back!

<img src="https://content.techgig.com/photo/80071467/pros-and-cons-of-python-programming-language-that-every-learner-must-know.jpg?132269" width="100%" style="margin-left:auto; margin-right:auto">